In [6]:
# --- START OF FILE Notebook3GPU-Models.ipynb ---

#######################################################################
# 0. Environment set‑up                                               #
#######################################################################
# ☑️  Install all dependencies.
#    Ensure your environment has GPU support for PyTorch if running on GPU nodes.
#    Note: 'neuralforecast' usually pulls 'pytorch-lightning' and 'torch'.
# !pip install --quiet pandas pyarrow matplotlib seaborn numpy scikit-learn tqdm \
#     torch pytorch-lightning neuralforecast

import os
import warnings
import random
import gc
import itertools
import time
from pathlib import Path
from typing import List, Dict, Tuple, Optional

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# For Data Scaling
from sklearn.preprocessing import StandardScaler
# For Model Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error

# PyTorch specific imports (for device management, though NeuralForecast handles much)
import torch

# For EarlyStopping in PyTorch Lightning (used by NeuralForecast internally)
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# For NeuralForecast models
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST

# Suppress minor warnings for cleaner output in Jupyter
warnings.filterwarnings("ignore")

# Ensure plots appear inline in Jupyter Notebook
# %matplotlib inline

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 150 # Increase resolution for better quality plots
plt.rcParams['savefig.dpi'] = 300 # Save plots with higher resolution

# Seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Check for GPU availability
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

#######################################################################
# 1. Configuration                                                    #
#######################################################################

# --- Paths & File Names ---
# Input: Directory containing your INDIVIDUAL processed hourly Parquet files (from Notebook 1)
INPUT_PROCESSED_HOURLY_DIR = Path("outputs/ch4") 

# Output root directory for results (figures, metrics, predictions)
OUT_DIR           = Path("outputs/ch4").absolute(); OUT_DIR.mkdir(parents=True, exist_ok=True)
PREDICTIONS_DIR = OUT_DIR / "predictions"; PREDICTIONS_DIR.mkdir(exist_ok=True, parents=True)
METRICS_FILE_PATH = OUT_DIR / "all_model_results.csv" # Path to the aggregated metrics CSV
LOSS_PNG_DIR = OUT_DIR / "loss_curves"          # <─ new

# --- Data Specifics ---
TARGET_COL        = "in_packets"           # The volume metric (packets)
ROUTER_COL        = "router"               # Column for router name
ANOM_SCORE_COL    = "if_score"             
ANOM_FLAG_COL     = "if_flag"              

# --- Time Series Parameters ---
LOOKBACK          = 336                     # Sliding window length (hours)
HORIZONS          = [1, 6, 12, 24]         # Prediction horizons
TEST_SIZE_HOURS   = 7 * 24                 # 7 days for testing
VAL_SIZE_HOURS    = 7 * 24                 # 7 days for validation (used for early stopping)

# --- Model Training Parameters ---
BATCH_SIZE        = 128
EPOCHS            = 400                    # Total epochs for training (subject to early stopping)
EARLY_STOP_PATIENCE =20                   # Patience for early stopping
LEARNING_RATE     = 1e-3
NF_MAX_STEPS      = EPOCHS                 # NeuralForecast uses max_steps instead of epochs
NF_VALIDATION_STEPS = 1 # Number of steps to check validation. Set to 1 for per-epoch check or longer for faster runs.
# NF_VAL_SIZE_FRACTION = 0.2 # This isn't directly used for the callbacks in NeuralForecast with 'val_check_steps' set to 1.
# NF uses `val_check_steps` if `val_loader` is present, or `limit_train_batches` if not.
# For simple EarlyStopping on `train_loss`, a dedicated validation set is not passed to NF.

# --- NeuralForecast specific parameters ---\
NF_LOOKBACK_SIZE = LOOKBACK 
NF_MODEL_MAX_HORIZON = max(HORIZONS) 
FREQ              = "h"                    # Hourly frequency for NeuralForecast

print(f"Configuration loaded.")
print(f"Input processed hourly data from: {INPUT_PROCESSED_HOURLY_DIR}")
print(f"Outputs will be saved to: {OUT_DIR} (predictions: {PREDICTIONS_DIR}, metrics: {METRICS_FILE_PATH})")
print(f"Time series lookback: {LOOKBACK} hours, horizons: {HORIZONS}")
print(f"Training Epochs (NeuralForecast max_steps): {NF_MAX_STEPS}, Early Stop Patience: {EARLY_STOP_PATIENCE}")
print(f"Using device: {DEVICE}")

#######################################################################
# 2. Evaluation Metrics (Expanded)                                    #
#######################################################################

def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    """
    Calculates MAE, RMSE, sMAPE, MSLE, and P99 Absolute Error.
    Ensures non-negative inputs for relevant metrics to avoid errors.
    """
    # Ensure y_true and y_pred are non-negative for log/percentage metrics
    y_true_safe = np.maximum(y_true, 0)
    y_pred_safe = np.maximum(y_pred, 0)
    
    mae = mean_absolute_error(y_true_safe, y_pred_safe)
    rmse = np.sqrt(mean_squared_error(y_true_safe, y_pred_safe))
    
    # sMAPE: Symmetric Mean Absolute Percentage Error
    # Add a small epsilon to the denominator to prevent division by zero
    smape_denominator = (np.abs(y_true_safe) + np.abs(y_pred_safe))
    if np.sum(smape_denominator) == 0:
        smape = 0.0
    else:
        smape = np.mean(2 * np.abs(y_pred_safe - y_true_safe) / (smape_denominator + 1e-8)) * 100
    
    # MSLE: Mean Squared Logarithmic Error
    # Add epsilon before log to prevent log(0) and handle cases where all values are zero
    if np.all(y_true_safe == 0) and np.all(y_pred_safe == 0):
        msle = 0.0 # If both are consistently zero, error is zero
    else:
        msle = mean_squared_log_error(y_true_safe + 1e-8, y_pred_safe + 1e-8) 
    
    # P99 Absolute Error
    p99_abs_error = np.percentile(np.abs(y_true_safe - y_pred_safe), 99)
    
    return {
        "MAE": mae,
        "RMSE": rmse,
        "sMAPE": smape,
        "MSLE": msle,
        "P99_abs_error": p99_abs_error
    }

# ─── Lightning callback: record loss each epoch & save a PNG ───
class LossCurveSaver(pl.callbacks.Callback):
    """Collects train_loss each epoch and writes loss_<variant>.png when fit() ends."""
    def __init__(self, variant_key: str, out_dir: Path):
        super().__init__()
        self.variant_key = variant_key
        self.out_dir     = out_dir
        self.history     = []

    def on_train_epoch_end(self, trainer, pl_module):
        # Lightning puts the running average in 'train_loss'
        loss = trainer.callback_metrics.get("train_loss_epoch",
                                            trainer.callback_metrics.get("train_loss"))
        if loss is not None:
            self.history.append(float(loss))

    def on_train_end(self, trainer, pl_module):
        if not self.history:
            return
        self.out_dir.mkdir(parents=True, exist_ok=True)
        import matplotlib.pyplot as plt
        plt.figure()
        plt.plot(self.history, lw=2)
        plt.title(self.variant_key)
        plt.xlabel("epoch"); plt.ylabel("train loss")
        plt.tight_layout()
        png = self.out_dir / f"loss_{self.variant_key}.png"
        plt.savefig(png, dpi=200)
        plt.close()
        print(f"· loss curve saved → {png.relative_to(Path.cwd())}")
# ----------------------------------------------------------------


#######################################################################
# 3. Model Experiment Runner for NeuralForecast Models                #
#######################################################################

def run_nf_experiment(model_type: str, df_full_scaled: pd.DataFrame, train_df: pd.DataFrame,
                      test_df: pd.DataFrame, target_scaler: StandardScaler,
                      router_label: str, exp_config: Dict, all_results: List[Dict], freq_param: str):
    """
    Runs a NeuralForecast model (PatchTST) with anomaly integration strategies.
    `df_full_scaled` is the complete time series with scaled features, including anomalies, for cross-validation.
    `train_df` is the potentially masked/modified training split.
    """
    exp_name = exp_config['name'].replace("NF", model_type)
    print(f"\n    --- Running NeuralForecast Experiment: {exp_name} ({model_type}) ---")

    # Prepare training DataFrame for NeuralForecast (handling masking)
    nf_train_df = train_df.copy() 
    if exp_config['mask_out']:
        initial_rows = len(nf_train_df)
        nf_train_df = nf_train_df[nf_train_df[ANOM_FLAG_COL] == 0].copy()
        print(f"    Masked out {initial_rows - len(nf_train_df)} anomalous periods for training.")
    
    
    # callback objects -------------------------------------------------
    variant_key   = f"{model_type}_{exp_name}"
    curve_cb      = LossCurveSaver(variant_key, LOSS_PNG_DIR)   # <─ new
    early_stop_cb = EarlyStopping(monitor="train_loss",
                                  patience=EARLY_STOP_PATIENCE,
                                  mode="min",
                                  check_on_train_epoch_end=True)
    # -----------------------------------------------------------------
    
    # Prepare `nf_train_df` columns for NeuralForecast
    nf_train_df = nf_train_df.reset_index().rename(columns={nf_train_df.index.name: "ds", TARGET_COL + '_scaled': "y"})
    nf_train_df['unique_id'] = router_label
    
    # exogenous_features = []
    # # Only add exogenous features if use_score is True (PatchTST supports them)
    # if exp_config['use_score']: 
    #     exogenous_features.append(ANOM_SCORE_COL + '_scaled')
    
    # ---- inside run_nf_experiment before building the model ----
    if exp_config['use_score']:
        print(f"WARNING: {model_type} does not support historical exogenous vars. "
              f"Skipping experiment {exp_name}.")
        return            # <- just exit this experiment
    exogenous_features = []        # leave list empty
        
    nf_train_df = nf_train_df[['unique_id', 'ds', 'y'] + exogenous_features]

    # Check if training data is empty after masking or for other reasons
    if nf_train_df.empty:
        print(f"    Warning: Training data for {exp_name} is empty. Skipping training and evaluation.")
        return

    # Define model instance for PatchTST
    patchtst_model_instance = PatchTST(
        h=NF_MODEL_MAX_HORIZON, 
        input_size=NF_LOOKBACK_SIZE,
        # hist_exog_list=exogenous_features if exogenous_features else None, 
        max_steps=NF_MAX_STEPS, 
        learning_rate=LEARNING_RATE, 
        random_seed=SEED,
        scaler_type=None, # We use sklearn.StandardScaler externally
        # callbacks=[EarlyStopping(monitor="train_loss", patience=EARLY_STOP_PATIENCE, mode="min", check_on_train_epoch_end=True)],
        callbacks=[early_stop_cb, curve_cb],
        accelerator=str(DEVICE), # Convert torch.device to string 'cuda' or 'cpu'
        val_check_steps=NF_VALIDATION_STEPS, # How often to check validation loss
        batch_size=BATCH_SIZE,
        alias="PatchTST" 
    )

    nf_instance = NeuralForecast(models=[patchtst_model_instance], freq=freq_param) 

    print(f"    Fitting {model_type}...")
    start_time = time.time()
    nf_instance.fit(df=nf_train_df)
    train_duration = time.time() - start_time
    print(f"    {model_type} fitted. Duration: {train_duration:.2f} seconds.")

    # Perform cross-validation to get predictions for the test set
    # `df_full_for_nf` must include all historical data up to the end of the test set,
    # and all necessary features (target and exogenous).
    df_full_for_nf = df_full_scaled.reset_index().rename(columns={df_full_scaled.index.name: "ds", TARGET_COL + '_scaled': "y"})
    df_full_for_nf['unique_id'] = router_label
    
    nf_df_cols_full = ['unique_id', 'ds', 'y'] + (exogenous_features if exogenous_features else [])
    df_full_for_nf = df_full_for_nf[nf_df_cols_full]

    start_time = time.time()
    # NeuralForecast's cross_validation predicts for horizons from the end of each window.
    # It will automatically manage the sliding window for prediction.
    nf_predictions_df = nf_instance.cross_validation(
        df=df_full_for_nf,
        val_size=VAL_SIZE_HOURS, # Validation size for splitting data for cross-validation
        test_size=TEST_SIZE_HOURS, # Test size for final evaluation window
        n_windows=None  #Only one forecast window for the test set
    )
    
        # keep only this router and guarantee integer horizon
    # keep only this router
    nf_predictions_df = nf_predictions_df.query("unique_id == @router_label").copy()

    # ------------------------------------------------------------------
    # (NEW) rebuild horizon if the column is missing
    if 'horizon' not in nf_predictions_df.columns:
        # make sure we’re dealing with datetimes
        nf_predictions_df['cutoff'] = pd.to_datetime(nf_predictions_df['cutoff'])
        nf_predictions_df['ds']     = pd.to_datetime(nf_predictions_df['ds'])

        # horizon in hours: (forecast timestamp – cutoff timestamp)
        nf_predictions_df['horizon'] = (
            (nf_predictions_df['ds'] - nf_predictions_df['cutoff'])
            .dt.total_seconds()
            .div(3600)
            .astype(int)
        )
    # ------------------------------------------------------------------

    # (always) coerce to int so the equality test below works
    nf_predictions_df['horizon'] = nf_predictions_df['horizon'].astype(int)
    #     # ----------------------- NEW / FIX --------------------------------
    # # robust horizon reconstruction and dtype coercion
    # nf_predictions_df['cutoff'] = pd.to_datetime(nf_predictions_df['cutoff'])
    # nf_predictions_df['ds']     = pd.to_datetime(nf_predictions_df['ds'])
    # nf_predictions_df['horizon'] = (
    #         (nf_predictions_df['ds'] - nf_predictions_df['cutoff'])
    #         /  np.timedelta64(1, "h")
    # ).round().astype(int)                 # ★★ int dtype guaranteed
    # nf_predictions_df = nf_predictions_df.query("horizon >= 1")  # keep 1..h
    # # ------------------------------------------------------------------
    
    inference_duration = time.time() - start_time
    print(f"    {model_type} prediction done. Duration: {inference_duration:.2f} seconds.")

    # Re-create the 'horizon' column manually for NF_predictions_df if needed
    if 'horizon' not in nf_predictions_df.columns:
        print("  'horizon' column not found in NF predictions. Recreating it.")
        nf_predictions_df['cutoff'] = pd.to_datetime(nf_predictions_df['cutoff'])
        nf_predictions_df['ds'] = pd.to_datetime(nf_predictions_df['ds'])
        # Calculate horizon in hours
        nf_predictions_df['horizon'] = ((nf_predictions_df['ds'] - nf_predictions_df['cutoff']).dt.total_seconds() // 3600).astype(int)
        # Filter out horizon <= 0 (historical/current values) if any
        nf_predictions_df = nf_predictions_df[nf_predictions_df['horizon'] > 0].copy()
        print("  Horizon column recreated — sample values:", np.sort(nf_predictions_df["horizon"].unique()[:10]))


    # Evaluate and store results, and save predictions for later plotting
    for horizon in HORIZONS:
        nf_preds_for_horizon = nf_predictions_df[nf_predictions_df['horizon'] == horizon]
        
        y_true_original_h = np.array([])
        y_pred_original_h = np.array([])
        m = {k: np.nan for k in ["MAE", "RMSE", "sMAPE", "MSLE", "P99_abs_error"]} # Initialize metrics to NaN

        if nf_preds_for_horizon.empty:
            print(f"    No predictions available for Horizon {horizon}h after filtering. Setting metrics to NaN.")
        else:
            y_true_scaled_h = nf_preds_for_horizon['y'].values
            y_pred_scaled_h = nf_preds_for_horizon[model_type].values # Predictions are in a column named after the model_type
            
            y_true_original_h = target_scaler.inverse_transform(y_true_scaled_h.reshape(-1, 1)).flatten()
            y_pred_original_h = target_scaler.inverse_transform(y_pred_scaled_h.reshape(-1, 1)).flatten()
            
            m = calculate_metrics(y_true_original_h, y_pred_original_h)

        # Save predictions for this horizon
        if y_pred_original_h.size > 0 and y_true_original_h.size > 0: # Check for non-empty arrays
            predictions_df = pd.DataFrame({
                'ds': nf_preds_for_horizon['ds'].values, # Timestamps from NF's output
                'y_true': y_true_original_h,
                'y_pred': y_pred_original_h
            })
            predictions_path = PREDICTIONS_DIR / f"{model_type}_{exp_name}_H{horizon}_predictions_{router_label}.parquet"
            predictions_df.to_parquet(predictions_path, index=False)
            print(f"      Saved predictions for {exp_name} H{horizon} to {predictions_path.name}")
        else:
            print(f"      Skipping saving predictions for {exp_name} H{horizon} due to empty or mis-aligned data.")

        print(f"    Horizon {horizon}h Metrics: MAE={m['MAE']:.2f}, RMSE={m['RMSE']:.2f}, sMAPE={m['sMAPE']:.2f}%, MSLE={m['MSLE']:.4f}, P99_abs_error={m['P99_abs_error']:.2f}")
        
        result_row = {
            "model": model_type,
            "experiment": exp_name,
            "horizon": horizon,
            "router": router_label,
            "train_time_sec": train_duration,
            "inference_time_sec": inference_duration,
            **m
        }
        all_results.append(result_row)
    
    # Clean up memory
    del patchtst_model_instance, nf_instance, nf_train_df, df_full_for_nf, nf_predictions_df
    gc.collect()


#######################################################################
# 4. Main Execution for Notebook 3                                    #
#######################################################################

def main():
    print("\n--- Initiating Notebook 3: GPU-Friendly Model Training & Evaluation (PatchTST) ---")

    # 1. Load all individual processed hourly Parquet files
    print(f"\nLoading all processed hourly data from: {INPUT_PROCESSED_HOURLY_DIR}/*.parquet")
    all_processed_hourly_files = sorted(list(INPUT_PROCESSED_HOURLY_DIR.glob("hourly_*_processed_with_anomalies.parquet")))
    
    if not all_processed_hourly_files:
        print(f"Error: No processed hourly Parquet files found in {INPUT_PROCESSED_HOURLY_DIR}. Please run Notebook 1 first.")
        return

    # Load all into a single Pandas DataFrame (feasible as these are small files)
    list_df_hourly = []
    for file_path in tqdm(all_processed_hourly_files, desc="Loading hourly files"):
        try:
            df = pd.read_parquet(file_path)
            # Ensure router column is present and correct (it should be from Notebook 1)
            router_label_from_file = file_path.stem.replace('hourly_', '').replace('_processed_with_anomalies', '')
            if ROUTER_COL not in df.columns:
                df[ROUTER_COL] = router_label_from_file
            list_df_hourly.append(df)
        except Exception as e:
            print(f"Warning: Could not load {file_path.name}: {e}. Skipping this file.")
            continue
    
    if not list_df_hourly:
        print("No valid hourly processed data loaded. Exiting.")
        return

    df_combined_hourly = pd.concat(list_df_hourly, ignore_index=False) # Keep original index (timestamps)
    print(f"Loaded combined hourly data: {len(df_combined_hourly)} rows from {len(list_df_hourly)} routers.")
    del list_df_hourly # Free memory
    gc.collect()

    # Get unique routers for iteration
    unique_routers = df_combined_hourly[ROUTER_COL].unique().tolist()
    unique_routers.sort()
    print(f"Found {len(unique_routers)} unique routers: {unique_routers}")

    # Initialize master results list
    all_results = []

    # Try to load existing results to append to them
    if METRICS_FILE_PATH.exists():
        try:
            existing_results_df = pd.read_csv(METRICS_FILE_PATH)
            all_results.extend(existing_results_df.to_dict('records'))
            print(f"Loaded {len(existing_results_df)} existing results from {METRICS_FILE_PATH}.")
        except Exception as e:
            print(f"Warning: Could not load existing metrics file {METRICS_FILE_PATH}: {e}. Starting with empty results.")
    else:
        print(f"No existing metrics file found at {METRICS_FILE_PATH}. Starting with empty results.")


    # Iterate through each router for processing
    for router_idx, router_label in enumerate(tqdm(unique_routers, desc="Processing Routers for Modeling (PatchTST)")):
        print(f"\n==== Starting Model Training & Evaluation for Router: {router_label} ({router_idx + 1}/{len(unique_routers)}) ====")
        
        # Filter data for current router
        df_hourly_processed_single_router = df_combined_hourly[df_combined_hourly[ROUTER_COL] == router_label].copy()
        
        if df_hourly_processed_single_router.empty:
            print(f"  Warning: No hourly data found for {router_label}. Skipping this router.")
            continue

        # Safeguard: Ensure no NaNs in critical columns
        initial_rows = len(df_hourly_processed_single_router)
        df_hourly_processed_single_router.dropna(subset=[TARGET_COL, ANOM_SCORE_COL], inplace=True)
        if len(df_hourly_processed_single_router) < initial_rows:
            print(f"  Dropped {initial_rows - len(df_hourly_processed_single_router)} rows with NaNs in target/anomaly columns for {router_label}.")
        
        # Time Series Splitting
        print("\n  --- Splitting Data into Train/Validation/Test Sets ---")
        total_len = len(df_hourly_processed_single_router)
        
        min_data_required = TEST_SIZE_HOURS + VAL_SIZE_HOURS + LOOKBACK + max(HORIZONS)
        if total_len < min_data_required:
            print(f"  Warning: Not enough data for requested split sizes and lookback/horizons for {router_label}. Total: {total_len} hours. Needed at least: {min_data_required} hours. Skipping this router.")
            continue

        test_start_idx = total_len - TEST_SIZE_HOURS
        val_start_idx = test_start_idx - VAL_SIZE_HOURS 
        
        # Train, Validation, Test splits for SCALING and for NF cross-validation
        train_df = df_hourly_processed_single_router.iloc[:val_start_idx].copy()
        val_df = df_hourly_processed_single_router.iloc[val_start_idx:test_start_idx].copy()
        test_df = df_hourly_processed_single_router.iloc[test_start_idx:].copy()

        # Combine train, val, test for full scaled series needed by NeuralForecast cross_validation
        df_full_scaled_router = pd.concat([train_df, val_df, test_df], axis=0) # Concatenate only for current router

        print(f"  Train set size: {len(train_df)} hours (from {train_df.index.min()} to {train_df.index.max()})\n"
              f"  Validation set size: {len(val_df)} hours (from {val_df.index.min()} to {val_df.index.max()})\n"
              f"  Test set size: {len(test_df)} hours (from {test_df.index.min()} to {test_df.index.max()})")


        # Data Scaling
        print("\n  --- Scaling Time Series Data ---")
        target_scaler = StandardScaler()
        # Fit scaler on training data, transform all splits
        df_full_scaled_router[TARGET_COL + '_scaled'] = target_scaler.fit_transform(df_full_scaled_router[[TARGET_COL]])

        anomaly_score_scaler = StandardScaler()
        # Fit scaler on training data, transform all splits
        df_full_scaled_router[ANOM_SCORE_COL + '_scaled'] = anomaly_score_scaler.fit_transform(df_full_scaled_router[[ANOM_SCORE_COL]])
        print("  Data scaled successfully using StandardScaler.")

        # Re-derive train_df, val_df, test_df from scaled df_full_scaled_router
        train_df_scaled = df_full_scaled_router.iloc[:val_start_idx].copy()
        val_df_scaled = df_full_scaled_router.iloc[val_start_idx:test_start_idx].copy()
        test_df_scaled = df_full_scaled_router.iloc[test_start_idx:].copy()


        # --- Define PatchTST Experiments configurations ---
        patchtst_experiments_configs = [
            {"name": "PatchTST_raw",                "mask_out": False, "use_score": False},
            {"name": "PatchTST_mask_out",           "mask_out": True,  "use_score": False},
            {"name": "PatchTST_score_as_feature",   "mask_out": False, "use_score": True},
            {"name": "PatchTST_both",               "mask_out": True,  "use_score": True}
        ]

        # --- Loop through each PatchTST experiment for the current router ---
        # This is the corrected loop that defines `exp`
        for exp in patchtst_experiments_configs: 
            run_nf_experiment(
                model_type="PatchTST",
                df_full_scaled=df_full_scaled_router, # Pass the router-specific combined scaled df
                train_df=train_df_scaled, # Pass the scaled train_df
                test_df=test_df_scaled,   # Pass the scaled test_df
                target_scaler=target_scaler,
                router_label=router_label,
                exp_config=exp, # `exp` is now correctly defined here
                all_results=all_results,
                freq_param=FREQ
            )
            # gc.collect() # run_nf_experiment already cleans up its internal DFs. No need for an extra gc here.
        
        # Clean up router-specific DataFrames after all experiments for this router are done
        del df_hourly_processed_single_router, train_df, val_df, test_df, df_full_scaled_router, \
            train_df_scaled, val_df_scaled, test_df_scaled
        gc.collect()
    
    # --- Final Save of all results ---
    final_results_df = pd.DataFrame(all_results)
    # Drop duplicates to prevent re-runs from adding redundant rows
    final_results_df.drop_duplicates(
        subset=['model', 'experiment', 'horizon', 'router'], 
        inplace=True
    )
    print(f"\nRemoved {len(all_results) - len(final_results_df)} duplicate result rows.")

    final_results_df.to_csv(METRICS_FILE_PATH, index=False)
    print(f"\nAll model results (including PatchTST) saved to: {METRICS_FILE_PATH}")

    print("\n--- Notebook 3 Complete: GPU-Friendly Model Training & Evaluation (PatchTST) ---")

# Execute the main pipeline for this notebook
if __name__ == "__main__":
    main()

# --- END OF FILE Notebook3GPU-Models.ipynb ---

Using device: cuda
Configuration loaded.
Input processed hourly data from: outputs/ch4
Outputs will be saved to: /home/ramamurthy/mhnarfth/network_analysis/individually process kora/outputs/ch4 (predictions: /home/ramamurthy/mhnarfth/network_analysis/individually process kora/outputs/ch4/predictions, metrics: /home/ramamurthy/mhnarfth/network_analysis/individually process kora/outputs/ch4/all_model_results.csv)
Time series lookback: 336 hours, horizons: [1, 6, 12, 24]
Training Epochs (NeuralForecast max_steps): 400, Early Stop Patience: 20
Using device: cuda

--- Initiating Notebook 3: GPU-Friendly Model Training & Evaluation (PatchTST) ---

Loading all processed hourly data from: outputs/ch4/*.parquet


Loading hourly files: 100%|██████████| 10/10 [00:00<00:00, 572.63it/s]

Loaded combined hourly data: 12604 rows from 10 routers.


Found 10 unique routers: ['atlanta', 'batonrouge', 'boston', 'dallas', 'elpaso', 'jackson', 'jacksonville', 'louisville', 'phoenix', 'reno']
Loaded 480 existing results from /home/ramamurthy/mhnarfth/network_analysis/individually process kora/outputs/ch4/all_model_results.csv.


Processing Routers for Modeling (PatchTST):   0%|          | 0/10 [00:00<?, ?it/s][rank: 0] Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type              | Params | Mode 
-----------------------------------------------------------
0 | loss         | MAE               | 0      | train
1 | padder_train | ConstantPad1d     | 0      | train
2 | scaler       | TemporalNorm      | 0      | train
3 | model        | PatchTST_backbone | 534 K  | train
-----------------------------------------------------------
534 K     Trainable params
3         Non-trainable params
534 K     Total params
2.136     Total estimated model params size (MB)
90        Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


...

    PatchTST fitted. Duration: 5.71 seconds.
Epoch 0: 100%|██████████| 1/1 [00:00<00:00, 23.34it/s, v_num=1.06e+7, train_loss_step=0.0424]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  4.08it/s, v_num=1.06e+7, train_loss_step=0.0424, valid_loss=0.230, train_loss_epoch=0.0424]


Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 183.58it/s]
    PatchTST prediction done. Duration: 0.41 seconds.
      Saved predictions for PatchTST_mask_out H1 to PatchTST_PatchTST_mask_out_H1_predictions_reno.parquet
    Horizon 1h Metrics: MAE=256185397.78, RMSE=449011480.96, sMAPE=172.86%, MSLE=175.1591, P99_abs_error=1457552359.68
      Saved predictions for PatchTST_mask_out H6 to PatchTST_PatchTST_mask_out_H6_predictions_reno.parquet
    Horizon 6h Metrics: MAE=168596820.09, RMSE=219448829.73, sMAPE=154.50%, MSLE=127.8566, P99_abs_error=511942534.08
      Saved predictions for PatchTST_mask_out H12 to PatchTST_PatchTST_mask_out_H12_predictions_reno.parquet
    Horizon 12h Metrics: MAE=165058278.58, RMSE=215773287.93, sMAPE=151.66%, MSLE=121.8140, P99_abs_error=515290513.60
      Saved predictions for PatchTST_mask_out H24 to PatchTST_PatchTST_mask_out_H24_predictions_reno.parquet
    Horizon 24h Metrics: MAE=193001399.28, RMSE=226175599.96, sMAPE=165.79%, MSLE=140.

Processing Routers for Modeling (PatchTST): 100%|██████████| 10/10 [04:40<00:00, 28.00s/it]


Removed 0 duplicate result rows.

All model results (including PatchTST) saved to: /home/ramamurthy/mhnarfth/network_analysis/individually process kora/outputs/ch4/all_model_results.csv

--- Notebook 3 Complete: GPU-Friendly Model Training & Evaluation (PatchTST) ---


In [5]:
final_results_df.to_csv(METRICS_FILE_PATH, index=False)

NameError: name 'final_results_df' is not defined